# Pampers Only
Start by only considering one product and trying to determine the optimal strategy

In [32]:
using CSV, JuMP

avg_demand = CSV.read("AverageDemandForecast.csv")

lb_demand = CSV.read("LowerBoundDemandForecast.csv")

ub_demand = CSV.read("UpperBoundDemandForecast.csv")

pampers_avg = zeros(Float64, 52)
pampers_lb = zeros(Float64, 52)
pampers_ub = zeros(Float64, 52)
ub_avg_diff = zeros(Float64, 52)

for i in 2:length(names(avg_demand))
    pampers_avg[(i-2)*2 + 1] = avg_demand[i][1]/2
    pampers_avg[(i-2)*2 + 2] = avg_demand[i][1]/2
    
    pampers_lb[(i-2)*2 + 1] = lb_demand[i][1]/2
    pampers_lb[(i-2)*2 + 2] = lb_demand[i][1]/2
    
    pampers_ub[(i-2)*2 + 1] = ub_demand[i][1]/2
    pampers_ub[(i-2)*2 + 2] = ub_demand[i][1]/2
    
    ub_avg_diff[(i-2)*2 + 1] = pampers_ub[(i-2)*2 + 1] - pampers_avg[(i-2)*2 + 1]
    ub_avg_diff[(i-2)*2 + 2] = pampers_ub[(i-2)*2 + 2] - pampers_avg[(i-2)*2 + 2]
    
end


safety_stock_heur = maximum(ub_avg_diff)
typical_avg_demand = sum(pampers_avg)/52

production_capacity_pallets  = 40
units_per_pallet = 576
unsatisfied_cost_per_unit = 100

inv_per_unit_per_week = .5*7

possible_reorder_lead_time = [1, 2, 3, 4, 5]
prod_cost = [25, 7.5, 2.5, 1, .5]

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[32]:15
└ @ Core ./In[32]:15
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[32]:16
└ @ Core ./In[32]:16
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[32]:18
└ @ Core ./In[32]:18
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[32]:19
└ @ Core ./In[32]:19
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[32]:21
└ @ Core ./In[32]:21
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[32]:22
└ @ Core ./In[32]:2

5-element Array{Float64,1}:
 25.0
  7.5
  2.5
  1.0
  0.5

In [45]:
using CSV, JuMP, Cbc

#found an equation for reorder quantity, but it doesn't seem solid when actually applied
#sqrt(2* quant required * order cost/ carrying cost per item)
#so for right now, we're going to let that be a variable in the formulation

m = Model(with_optimizer(Cbc.Optimizer, logLevel = 0))

@variable(m, 0 <= reorder_lead_time <= 5, Int)
@variable(m, reorder_amt_pallets >= 0, Int)

#note that the model will also assume that I start at a reorder point
#my reorder point heuristic is avg demand*lead time + safety stock
@expression(m, reorder_point, typical_avg_demand*reorder_lead_time + safety_stock_heur)

#I'm also assuming for the sake of simplicity that we experience avg_demand
#and simulate what that looks like given a certain lead time, amount ordered, and reorder point
simulated_demand = pampers_avg

#running tab of how much we have on hand for every week
@expression(m, amt_on_hand[1:52], 0)
#binary variable to express whether or not an order of the reorder amount is placed in any given week
@expression(m, order_placed[1:52], 0)

@expression(m, storage_costs, 0)
@expression(m, shipping_costs, 0)
@expression(m, unmet_orders_costs, 0)

amt_on_hand[1] = reorder_point
order_placed[1] = 1

for week in 2:52
    #simulate recieving orders
    if week-reorder_lead_time > 0
        if order_placed[week-reorder_lead_time] == week
            amt_on_hand[week] += reorder_amt_pallets*units_per_pallet
        end
    end
    
    println("checkpt 2")
    
    #simulate failing to fulfil orders
    if amt_on_hand[week] <= simulated_demand[week]
        unmet_orders_cost += (simulated_demand[week]-amt_on_hand[week])*unsatisfied_cost_per_unit
        amt_on_hand[week] = 0
    end
    
    println("checkpt 3")
    
    #simulate fulfilling orders and having excess on hand
    if amt_on_hand[week] > simulated_demand[week]
        amt_on_hand[week] += -simulated_demand[week]
        storage_costs += (amt_on_hand[week])*inv_per_unit_per_week
    end
    
    println("checkpt 4")
    
    #simulate placing an order
    if amt_on_hand[week] <= reorder_point
        order_placed[week] = 1
        shipping_costs += reorder_amt_pallets*prod_cost[reorder_lead_time]
    end
    
    println("checkpt 5")
end

@objective(m, Min, storage_costs+shipping_costs+unmet_orders_costs)
optimize!(m)

println("Objective Value: ", JuMP.objective_value(m))

MethodError: MethodError: no method matching isless(::Int64, ::GenericAffExpr{Float64,VariableRef})
Closest candidates are:
  isless(!Matched::Missing, ::Any) at missing.jl:87
  isless(::Real, !Matched::AbstractFloat) at operators.jl:157
  isless(::Integer, !Matched::ForwardDiff.Dual{Ty,V,N} where N where V) where Ty at /Users/eobermaier/.julia/packages/ForwardDiff/cXTw0/src/dual.jl:140
  ...

In [57]:
#trying to almost hard code the optimization that i was trying to do above
#this is pretty gross though, and will only get worse when i try to incorporate other products
#treating revenue like negative costs

simulated_demand = pampers_avg

best_cost = -1
best_breakdown = -1
best_reorder_lead_time = -1
best_reorder_amt_pallets = -1

amt_on_hand = zeros(Float64, 52)
order_placed = zeros(Float64, 52)

for reorder_lead_time in 1:5
    for reorder_amt_pallets in 1:1000
        
        storage_costs = 0
        shipping_costs = 0
        unmet_orders_costs = 0
        revenue = 0
        
        reorder_point = typical_avg_demand*reorder_lead_time + safety_stock_heur
        
        amt_on_hand[1] = reorder_point
        order_placed[1] = 1
        
        #simulate week over week
        for week in 2:52
            #simulate recieving orders
            if week-reorder_lead_time > 0
                if order_placed[week-reorder_lead_time] == 1
                    amt_on_hand[week] += reorder_amt_pallets*units_per_pallet
                end
            end
            
            #simulate failing to fulfil orders
            if amt_on_hand[week] <= simulated_demand[week]
                revenue += amt_on_hand[week]*unsatisfied_cost_per_unit/2
                unmet_orders_costs += (simulated_demand[week]-amt_on_hand[week])*unsatisfied_cost_per_unit
                amt_on_hand[week] = 0
            end
            
            #simulate fulfilling orders and having excess on hand
            if amt_on_hand[week] > simulated_demand[week]
                revenue += simulated_demand[week]*unsatisfied_cost_per_unit/2
                amt_on_hand[week] += -simulated_demand[week]
                storage_costs += (amt_on_hand[week])*inv_per_unit_per_week
            end
            
            #simulate placing an order
            if amt_on_hand[week] <= reorder_point
                order_placed[week] = 1
                shipping_costs += reorder_amt_pallets*prod_cost[reorder_lead_time]
            end
            
        end
        
        #figure out the best option
        if best_cost == -1
            best_cost = storage_costs + shipping_costs + unmet_orders_costs - revenue
            best_breakdown = (storage_costs, shipping_costs, unmet_orders_costs, revenue)
            best_reorder_lead_time = reorder_lead_time
            best_reorder_amt_pallets = reorder_amt_pallets
        end
        
        if best_cost > storage_costs + shipping_costs + unmet_orders_costs - revenue
            best_cost = storage_costs + shipping_costs + unmet_orders_costs - revenue
            best_breakdown = (storage_costs, shipping_costs, unmet_orders_costs, revenue)
            best_reorder_lead_time = reorder_lead_time
            best_reorder_amt_pallets = reorder_amt_pallets
        end
        
    end
end

println(best_cost)
println(best_breakdown)
println("")
println(best_reorder_lead_time)
println(best_reorder_amt_pallets)

-4.42592545e7
(8.8046455e6, 32500.0, 6.4654e6, 5.95618e7)

1
50
